In [2]:
!pip install cvlib

  Created wheel for cvlib: filename=cvlib-0.2.7-py3-none-any.whl size=10046376 sha256=0b10a4add8750e43f37735a58bbfb1740440a762ac36f72f2ddb1767bc484027
  Stored in directory: c:\users\vijendra\appdata\local\pip\cache\wheels\8e\d7\31\bc643bd3a8b11a7368b1ab1d8a6299b33b462ed0b0683ddc5a
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12078 sha256=2291ac4f07f72e79db16c3a64a3a71b2a1a8889652b1d0b6bb1547bfb7db2126
  Stored in directory: c:\users\vijendra\appdata\local\pip\cache\wheels\f0\fd\1f\3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25862 sha256=e102a99f70cd50411a3365acb58696f3004ff3bd1565ae0236e7d8ff2b470fdd
  Stored in directory: c:\users\vijendra\appdata\local\pip\cache\wheels\86\d7\0a\4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built cvlib progressbar imutils


In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv

In [3]:
# load model
model = load_model('gender_detection1.model')


In [4]:
# open webcam
webcam = cv2.VideoCapture(0)
    
classes = ['man','woman']


In [ ]:
# loop through frames
while webcam.isOpened():

    # read frame from webcam 
    status, frame = webcam.read()

    # apply face detection
    face, confidence = cv.detect_face(frame)


    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region
        face_crop = np.copy(frame[startY:endY,startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0] # model.predict return a 2D matrix, eg: [[9.9993384e-01 7.4850512e-05]]

        # get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# release resources
webcam.release()
cv2.destroyAllWindows()

In [ ]:
# release resources
# webcam.release()
# cv2.destroyAllWindows()